### Prodigy ML Task 5 ###
Develop a model that can accurately recognize food items from images and estimate their calorie content, enabling users to track their dietary intake and make informed food choices.

In [2]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Set the dataset path
dataset_path = "C:/Study Material/Iternship/Prodigy/Task 5/archive/food-101/food-101/"

# Paths to meta files
train_file = os.path.join(dataset_path, "meta/train.txt")
test_file = os.path.join(dataset_path, "meta/test.txt")

# Function to load file paths from meta files
def load_file_paths(file_path):
    with open(file_path, "r") as file:
        return [line.strip() for line in file.readlines()]

train_paths = load_file_paths(train_file)
test_paths = load_file_paths(test_file)

# Create a dictionary to map image paths to class names
def create_class_mapping(paths):
    class_mapping = {}
    for path in paths:
        class_name = path.split('/')[0]
        class_mapping[path] = class_name
    return class_mapping

train_mapping = create_class_mapping(train_paths)
test_mapping = create_class_mapping(test_paths)

# Load classes and labels
with open(os.path.join(dataset_path, "meta/classes.txt"), "r") as file:
    class_names = [line.strip() for line in file.readlines()]

num_classes = len(class_names)

# Data Augmentation
batch_size = 32
image_size = (160, 160)  # Consider resizing further if necessary

datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=15,  # Adjusted rotation range
    width_shift_range=0.1,  # Adjusted shift range
    height_shift_range=0.1,  # Adjusted shift range
    shear_range=0.1,  # Adjusted shear range
    zoom_range=0.1,  # Adjusted zoom range
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    fill_mode="nearest",
    validation_split=0.2  # Set aside 20% of data for validation
)

train_generator = datagen.flow_from_directory(
    os.path.join(dataset_path, "images"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    os.path.join(dataset_path, "images"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Load pre-trained ResNet50 model + higher level layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Adding custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,  # Start with more epochs, early stopping will handle the rest
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Calorie estimation dictionary (Example values, modify as needed)
calorie_dict = {
    'Apple Pie': '300 calories per slice',
    'Baby Back Ribs': '500 calories per serving',
    'Baklava': '350 calories per piece',
    'Beef Carpaccio': '150 calories per serving',
    'Beef Tartare': '200 calories per serving',
    'Beet Salad': '150 calories per serving',
    'Beignets': '250 calories per piece',
    'Bibimbap': '550 calories per bowl',
    'Bread Pudding': '400 calories per serving',
    'Breakfast Burrito': '600 calories per burrito',
    'Bruschetta': '150 calories per piece',
    'Caesar Salad': '350 calories per serving',
    'Cannoli': '250 calories per piece',
    'Caprese Salad': '300 calories per serving',
    'Carrot Cake': '350 calories per slice',
    'Ceviche': '200 calories per serving',
    'Cheesecake': '450 calories per slice',
    'Cheese Plate': '600 calories per plate',
    'Chicken Curry': '450 calories per serving',
    'Chicken Quesadilla': '500 calories per serving',
    'Chicken Wings': '430 calories per serving (about 6 wings)',
    'Chocolate Cake': '500 calories per slice',
    'Chocolate Mousse': '350 calories per serving',
    'Churros': '250 calories per serving (about 2-3 churros)',
    'Clam Chowder': '200 calories per bowl',
    'Club Sandwich': '600 calories per sandwich',
    'Crab Cakes': '300 calories per serving (about 2 cakes)',
    'Creme Brulee': '300 calories per serving',
    'Croque Madame': '550 calories per sandwich',
    'Cup Cakes': '350 calories per cupcake',
    'Deviled Eggs': '200 calories per 2 halves',
    'Donuts': '300 calories per donut',
    'Dumplings': '250 calories per serving (about 6 dumplings)',
    'Edamame': '120 calories per serving (about 1 cup)',
    'Eggs Benedict': '400 calories per serving',
    'Escargots': '250 calories per serving',
    'Falafel': '350 calories per serving (about 3 pieces)',
    'Filet Mignon': '450 calories per serving (about 6 oz)',
    'Fish and Chips': '600 calories per serving',
    'Foie Gras': '400 calories per serving',
    'French Fries': '300 calories per serving (about 1 cup)',
    'French Onion Soup': '300 calories per bowl',
    'French Toast': '400 calories per serving (about 2 slices)',
    'Fried Calamari': '300 calories per serving',
    'Fried Rice': '400 calories per serving',
    'Frozen Yogurt': '200 calories per serving (about 1/2 cup)',
    'Garlic Bread': '150 calories per slice',
    'Gnocchi': '250 calories per serving',
    'Greek Salad': '200 calories per serving',
    'Grilled Cheese Sandwich': '400 calories per sandwich',
    'Grilled Salmon': '350 calories per serving (about 6 oz)',
    'Guacamole': '250 calories per serving (about 1/2 cup)',
    'Gyoza': '200 calories per serving (about 5 pieces)',
    'Hamburger': '500 calories per burger',
    'Hot and Sour Soup': '150 calories per bowl',
    'Hot Dog': '300 calories per hot dog',
    'Huevos Rancheros': '550 calories per serving',
    'Hummus': '180 calories per serving (about 1/4 cup)',
    'Ice Cream': '200 calories per serving (about 1/2 cup)',
    'Lasagna': '600 calories per serving',
    'Lobster Bisque': '350 calories per bowl',
    'Lobster Roll Sandwich': '400 calories per sandwich',
    'Macaroni and Cheese': '450 calories per serving',
    'Macarons': '150 calories per macaron',
    'Miso Soup': '100 calories per bowl',
    'Mussels': '250 calories per serving',
    'Nachos': '600 calories per serving',
    'Omelette': '300 calories per omelette',
    'Onion Rings': '400 calories per serving',
    'Oysters': '100 calories per serving (about 6 oysters)',
    'Pad Thai': '600 calories per serving',
    'Paella': '500 calories per serving',
    'Pancakes': '350 calories per serving (about 2 pancakes)',
    'Panna Cotta': '300 calories per serving',
    'Peking Duck': '600 calories per serving',
    'Pho': '450 calories per bowl',
    'Pizza': '300 calories per slice',
    'Pork Chop': '400 calories per serving (about 6 oz)',
    'Poutine': '600 calories per serving',
    'Prime Rib': '750 calories per serving (about 10 oz)',
    'Pulled Pork Sandwich': '500 calories per sandwich',
    'Ramen': '450 calories per bowl',
    'Ravioli': '350 calories per serving',
    'Red Velvet Cake': '400 calories per slice',
    'Risotto': '400 calories per serving',
    'Samosa': '200 calories per piece',
    'Sashimi': '150 calories per serving',
    'Scallops': '250 calories per serving',
    'Shrimp Cocktail': '200 calories per serving',
    'Sushi': '200 calories per serving (about 8 pieces)',
    'Tacos': '300 calories per taco',
    'Tamales': '400 calories per serving',
    'Tiramisu': '350 calories per serving',
    'Tom Yum Soup': '150 calories per bowl',
    'Vegetable Soup': '100 calories per bowl',
    'Waffles': '350 calories per serving (about 2 waffles)',
    'Wings': '400 calories per serving (about 6 wings)'
}

# Example usage
def estimate_calories(food_item):
    return calorie_dict.get(food_item, "Calorie information not available")

# Test the calorie estimation function
print(estimate_calories('Pizza'))  # Example test



Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.
Epoch 1/20
2525/2525 [==============================] - 1982s 784ms/step - loss: 4.5947 - accuracy: 0.0150 - val_loss: 4.5620 - val_accuracy: 0.0171 - lr: 1.0000e-04
Epoch 2/20
2525/2525 [==============================] - 1951s 773ms/step - loss: 4.5446 - accuracy: 0.0231 - val_loss: 4.5284 - val_accuracy: 0.0241 - lr: 1.0000e-04
Epoch 3/20
2525/2525 [==============================] - 1950s 772ms/step - loss: 4.5143 - accuracy: 0.0276 - val_loss: 4.5020 - val_accuracy: 0.0317 - lr: 1.0000e-04
Epoch 4/20
2525/2525 [==============================] - 1967s 779ms/step - loss: 4.4874 - accuracy: 0.0315 - val_loss: 4.4776 - val_accuracy: 0.0325 - lr: 1.0000e-04
Epoch 5/20
2525/2525 [==============================] - 1944s 770ms/step - loss: 4.4638 - accuracy: 0.0347 - val_loss: 4.4609 - val_accuracy: 0.0359 - lr: 1.0000e-04
Epoch 6/20
2525/2525 [==============================] - 1945s 770ms/step - loss:

In [ ]:
# Unfreeze some layers of the base model for incremental learning
for layer in base_model.layers[-30:]:  # Unfreeze last 30 layers, adjust as needed
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training the model with additional epochs
additional_epochs = 25
history_incremental = model.fit(
    train_generator,
    epochs=additional_epochs,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping]
)



Since the model's accuracy was no longer improving I had to manually stop the training and after stopping the training I saved the model

In [7]:

# Save the updated model after additional training
model.save('incremental_food_recognition_model.keras')


Additionally I trained it for further more epochs to check the accuracy

In [8]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

# Load the previously saved model
model = load_model('incremental_food_recognition_model.keras')

# Optionally unfreeze more layers for fine-tuning
for layer in model.layers[-30:]:  # Unfreeze the last 30 layers, adjust as needed
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training the model with additional epochs
additional_epochs = 1  # Set the number of additional epochs
history = model.fit(
    train_generator,
    epochs=additional_epochs,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Save the model again after additional training
model.save('incremental_food_recognition_model_v2.keras')


2525/2525 [==============================] - 3216s 1s/step - loss: 3.4703 - accuracy: 0.1898 - val_loss: 3.6408 - val_accuracy: 0.1626 - lr: 1.0000e-05
